In [2]:
import scipy.io as io
import os
import numpy as np
import pandas as pd
import shutil
import io
import string
import re
import math
import struct

In [3]:
ham_part3 = pd.read_csv('raw_data/ham_part3.csv')
ham_part3.head(20)

,Input.label,Input.text,Answer.Q1Answer,Answer.html_output
0,1,The food quality and portion size was awesome....,yes,"<span>The</span> <span class=""active"">food</sp..."
1,1,The food quality and portion size was awesome....,yes,<span>The</span> <span>food</span> <span class...
2,1,The food quality and portion size was awesome....,yes,<span>The</span> <span>food</span> <span>quali...
3,1,Found these guys at taste of calgary and had t...,yes,<span>Found</span> <span>these</span> <span>gu...
4,1,Found these guys at taste of calgary and had t...,yes,<span>Found</span> <span>these</span> <span>gu...
5,1,Found these guys at taste of calgary and had t...,yes,<span>Found</span> <span>these</span> <span>gu...
6,1,Love the bar on the roof. Nice and relaxing f...,yes,"<span class=""active"">Love</span> <span>the</sp..."
7,1,Love the bar on the roof. Nice and relaxing f...,yes,"<span class=""active"">Love</span> <span class=""..."
8,1,Love the bar on the roof. Nice and relaxing f...,yes,"<span class=""active"">Love</span> <span class=""..."
9,1,Food is awesome. Service is great. What a team...,yes,<span>Food</span> <span>is</span> <span class=...


In [4]:
for i in range(1000):
    if (ham_part3['Input.text'][i*3] != ham_part3['Input.text'][i*3+1]) or (ham_part3['Input.text'][i*3] != ham_part3['Input.text'][i*3+2]) or (ham_part3['Input.text'][i*3+2] != ham_part3['Input.text'][i*3+1]):
        print(i)
    if (ham_part3['Input.label'][i*3] != ham_part3['Input.label'][i*3+1]) or (ham_part3['Input.label'][i*3] != ham_part3['Input.label'][i*3+2]) or (ham_part3['Input.label'][i*3+2] != ham_part3['Input.label'][i*3+1]):
        print(i)

In [5]:
def generate_binary_human_attention_vector(html, num_words_in_review, max_words):

    p = re.compile('<span(.*?)/span>')
    all_span_items = p.findall(html)
    
    if html == '{}':
        print('Empty human annotation - This should never print')
        return [0] * max_words
    
    if (len(all_span_items) == num_words_in_review+1 ):
        if ((all_span_items[num_words_in_review] == '><') or (all_span_items[num_words_in_review] == ' data-vivaldi-spatnav-clickable="1"><')):
            
            binarized_human_attention = [0] * max_words
            for i in range(0,len(all_span_items)-1):
                if 'class="active"' in all_span_items[i]:
                    binarized_human_attention[i] = 1

        else:
            print('This should never print.')
    else:
        print('This should never print.')

        
    return binarized_human_attention

def clean_document(document):
    document = document.lower()
    table = str.maketrans({key: None for key in string.punctuation})
    cleaned_document = document.translate(table)  
    return cleaned_document

In [23]:
lens_list = [len(ham_part3['Input.text'][i].split()) for i in range(len(ham_part3))]
print(max(lens_list))

73


In [24]:
human_intersections = []
cleaned_reviews = []
labels = []
MAX_WORDS = 73
for i in range(1000):
    review = clean_document(ham_part3['Input.text'][i*3])
    num_words_in_review = len(ham_part3['Input.text'][i*3].split())
    hatt1 = ham_part3['Answer.html_output'][i*3]
    hatt2 = ham_part3['Answer.html_output'][i*3+1]
    hatt3 = ham_part3['Answer.html_output'][i*3+2]
    b_hatt1 = np.array(generate_binary_human_attention_vector(hatt1, num_words_in_review, MAX_WORDS))
    b_hatt2 = np.array(generate_binary_human_attention_vector(hatt2, num_words_in_review, MAX_WORDS))
    b_hatt3 = np.array(generate_binary_human_attention_vector(hatt3, num_words_in_review, MAX_WORDS))
    b_inter = np.min(np.array([b_hatt1, b_hatt2, b_hatt3]),axis=0)
    human_intersections.append(b_inter)
    cleaned_reviews.append(review)
    labels.append(ham_part3['Input.label'][i*3].item())

In [25]:
total_instance = len(cleaned_reviews)
random_posits = np.arange(total_instance)
np.random.shuffle(random_posits)

#Turn into numpy arrays 
labels          = np.array(labels)
cleaned_reviews = np.array(cleaned_reviews)
human_intersections = np.array(human_intersections)

labels          = labels[random_posits]
cleaned_reviews = cleaned_reviews[random_posits]
human_intersections = human_intersections[random_posits]

In [28]:
sl = 547
print(cleaned_reviews[sl])
print(human_intersections[sl])
print(labels[sl])

as i am sitting here 2 other people here i have all u can eat pancakes send in to get more hear from the back i just made them why should i make more i guess the reviews on here are right this place is as bad in service the food was good but we will not be back here again  and chuck the manger was rude like i did some thing wrong
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
0
as i am sitting here 2 other people here i have all u can eat pancakes send in to get more hear from the back i just made them why should i make more i guess the reviews on here are right this place is as bad in service the food was good but we will not be back here again  and chuck the manger was rude like i did some thing wrong
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
0
695


In [37]:
embeddings_index = dict()
with open('data/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [40]:
max_words = 73 #num_words

output_reviews = []
for review in cleaned_reviews:
    words = review.split()
    words_mapped = [0]* max_words
    
    length = len(words)
    if (length<max_words):
        #print('We should never see this print')
        for i in range(0,length):
            words_mapped[i] = embeddings_index.get(words[i], embeddings_index['unk'])
        for i in range(length,max_words):
            words_mapped[i] =  embeddings_index['unk']
    elif (length>max_words):
        print('We should never see this print either')
    else:
        for i in range(0,max_words):
            words_mapped[i] = embeddings_index.get(words[i], embeddings_index['unk'])
            
    output_reviews.append(words_mapped)
    
output_reviews = np.array(output_reviews)
print(output_reviews.shape)
print(output_reviews[0])

(1000, 73, 100)
[[-0.17791    0.62675    0.4787    ... -0.19526    0.20078    1.0953   ]
 [ 0.27455   -0.0093143  0.50753   ...  0.36132    0.23673    0.066652 ]
 [-0.43061    0.28297    0.80489   ... -0.48895    0.50908    0.60909  ]
 ...
 [ 0.027166  -0.1762    -0.19623   ... -0.37226   -0.28782   -0.015834 ]
 [ 0.027166  -0.1762    -0.19623   ... -0.37226   -0.28782   -0.015834 ]
 [ 0.027166  -0.1762    -0.19623   ... -0.37226   -0.28782   -0.015834 ]]


In [44]:
np.save('data/x_train.npy', output_reviews)
np.save('data/y_train.npy', labels)
np.save('data/human_intersection.npy', human_intersections)
np.save('data/raw_reviews.npy', cleaned_reviews)